# Import Dependencies

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import os
import json
import requests

# clean_fl_ccd_directory DataFrame

In [3]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/ccd/directory/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_directory_response = requests.get(query_url)
fl_ccd_directory_data = fl_ccd_directory_response.json()

# Establish the results from the request to loop through below
results_directory = fl_ccd_directory_data['results']

# Create Lists to store values for the columns
district_name = []
dir_nces_id = []
state_id = []
latitude = []
longitude = []
urban_locale = []
agency_type = []
total_schools = []
total_students = []
spec_ed_stud = []
eng_lang_stud = []
teachers_fte = []
instru_aids_fte = []
guidance_cons_fte = []
school_staff_fte = []
district_staff_fte = []
staff_total_fte = []

# Loop through results and store values in appropriate lists
for result in results_directory:
    district_name.append(result['lea_name'])
    dir_nces_id.append(result['leaid'])
    state_id.append(result['state_leaid'])
    latitude.append(result['latitude'])
    longitude.append(result['longitude'])
    urban_locale.append(result['urban_centric_locale'])
    agency_type.append(result['agency_type'])
    total_schools.append(result['number_of_schools'])
    total_students.append(result['enrollment'])
    spec_ed_stud.append(result['spec_ed_students'])
    eng_lang_stud.append(result['english_language_learners'])
    teachers_fte.append(result['teachers_total_fte'])
    instru_aids_fte.append(result['instructional_aides_fte'])
    guidance_cons_fte.append(result['guidance_counselors_total_fte'])
    school_staff_fte.append(result['school_staff_total_fte'])
    district_staff_fte.append(result['lea_staff_total_fte'])
    staff_total_fte.append(result['staff_total_fte'])
    
# Create dataframe from results
fl_ccd_directory = pd.DataFrame({"District Name":district_name, "NCES ID":dir_nces_id, "State ID":state_id,                          
                                 "Degree of urbanization":urban_locale, "Agency Type":agency_type,
                                "Total Schools":total_schools, "Total Students Enrolled":total_students,
                                "Special Ed Students":spec_ed_stud, "English Language Students":eng_lang_stud,
                                "Total FTE Teachers":teachers_fte, "Total FTE Instructional Aids": instru_aids_fte,
                                "Total FTE Guidance Counselors": guidance_cons_fte, "Total FTE School Staff":school_staff_fte,
                                "Total FTE District Staff": district_staff_fte, "Total FTE Staff": staff_total_fte,
                                 "Latitude":latitude, "Longitude":longitude})

# Remove all districts that are not regular local school district
clean_fl_ccd_directory = fl_ccd_directory.loc[fl_ccd_directory['Agency Type'] == 1]


# Format columns
# clean_fl_ccd_directory = clean_fl_ccd_directory.style.format({"Total Students Enrolled": "{:,}" , "Special Ed Students": "{:,}",
#                                                             "English Language Students":"{:,}"})

clean_fl_ccd_directory

,District Name,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,English Language Students,Total FTE Teachers,Total FTE Instructional Aids,Total FTE Guidance Counselors,Total FTE School Staff,Total FTE District Staff,Total FTE Staff,Latitude,Longitude
2,ALACHUA,1200030,01,12,1,73,28689,3788,584,1833,535,61,None,None,4249,29.652601,-82.318977
3,BAKER,1200060,02,32,1,9,4936,538,11,301,65,10,None,None,604,30.277151,-82.118149
5,BAY,1200090,03,22,1,51,27641,4575,567,1844,732,66,None,None,3735,30.172516,-85.675934
6,BRADFORD,1200120,04,32,1,12,3199,645,6,242,56,8,None,None,507,29.951584,-82.112663
7,BREVARD,1200150,05,21,1,129,72285,11931,2344,4776,791,183,None,None,9109,28.248600,-80.736115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,UNION,1201890,63,32,1,9,2384,387,0,175,32,6,None,None,343,30.017477,-82.338341
66,VOLUSIA,1201920,64,21,1,92,61777,9879,3479,4153,593,142,None,None,7586,29.030180,-81.308472
67,WAKULLA,1201950,65,42,1,15,5141,976,8,327,84,9,None,None,708,30.176279,-84.379211
68,WALTON,1201980,66,41,1,23,8435,799,300,569,99,18,None,None,1164,30.720671,-86.098549


# drop_ids List

In [4]:
# Create a list of district names that were not regular local schools to drop from other tables
drop_agencies = fl_ccd_directory[fl_ccd_directory['Agency Type'] != 1]
drop_ids = drop_agencies['District Name'].tolist()

drop_ids

['FL VIRTUAL',
 'CESA',
 'LAKE WALES CHARTER SCHOOLS',
 'WASH SPECIAL',
 'FAU LAB SCH',
 'FSU LAB SCH',
 'FAMU LAB SCH',
 'UF LAB SCH',
 'DEAF/BLIND']

# clean_fl_ccd_financial DataFrame

In [5]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/ccd/finance/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_finance_response = requests.get(query_url)
fl_ccd_finance_data = fl_ccd_finance_response.json()

# Establish the results from the request to loop through below
results_financial = fl_ccd_finance_data['results']

# Create Lists to store values for the columns
fin_nces_id = []
total_rev = []
fed_total_rev = []
st_total_rev = []
loc_total_rev = []
loc_prop_tax_rev = []
intruct_staff_exp = []
general_admin_exp = []
school_admin_exp = []
operation_plant_exp = []
student_transport_exp = []

# Loop through results and store values in appropriate lists
for result in results_financial:
    fin_nces_id.append(result['leaid'])
    total_rev.append(result['rev_total'])
    fed_total_rev.append(result['rev_fed_total'])
    st_total_rev.append(result['rev_state_total'])
    loc_total_rev.append(result['rev_local_total'])
    loc_prop_tax_rev.append(result['rev_local_prop_tax'])
    intruct_staff_exp.append(result['exp_current_instruc_staff'])
    general_admin_exp.append(result['exp_current_general_admin'])
    school_admin_exp.append(result['exp_current_sch_admin'])
    operation_plant_exp.append(result['exp_current_operation_plant'])
    student_transport_exp.append(result['exp_current_student_transport'])
    
# Create dataframe from results
fl_ccd_financial = pd.DataFrame({"NCES ID":fin_nces_id, "Total Revenue":total_rev, "Total Federal Revenue":fed_total_rev,                        
                                 "Total State Revenue":st_total_rev, "Total Local Revenue":loc_total_rev,
                                "Local Revenue (Property Tax)":loc_prop_tax_rev, "Total Expense for Instruction Staff":intruct_staff_exp,
                                "Total Expense for General Admin":general_admin_exp, "Total Expense for School Admin":school_admin_exp,
                                "Total Expense for Operations":operation_plant_exp, "Total Expense for Student Transportation":student_transport_exp})

# Remove all districts without financial data
clean_fl_ccd_financial = fl_ccd_financial.loc[fl_ccd_financial['Total Revenue'] > 0]



# Format columns
# clean_fl_ccd_financial = clean_fl_ccd_financial.style.format({"Total Revenue": "${:,.0f}" , "Total Federal Revenue": "${:,.0f}",
#                                                             "Total State Revenue":"${:,.0f}", "Total Local Revenue":"${:,.0f}",
#                                                             "Local Revenue (Property Tax)":"${:,.0f}", "Total Expense for Instruction Staff":"${:,.0f}",
#                                                             "Total Expense for General Admin":"${:,.0f}", "Total Expense for School Admin":"${:,.0f}",
#                                                             "Total Expense for Operations":"${:,.0f}", "Total Expense for Student Transportation":"${:,.0f}"})

clean_fl_ccd_financial

,NCES ID,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
2,1200030,272451000.0,34881000.0,119810000.0,117760000,103083000,21763000,3041000,13128000,29535000,11174000
3,1200060,42674000.0,4704000.0,29197000.0,8773000,6266000,2595000,732000,2007000,4385000,2613000
5,1200090,268131000.0,26772000.0,103527000.0,137832000,101316000,11183000,2045000,13550000,20567000,8132000
6,1200120,31053000.0,4941000.0,17021000.0,9091000,6476000,1490000,734000,1792000,3197000,1629000
7,1200150,673236000.0,76493000.0,320008000.0,276735000,221801000,49626000,5282000,36220000,55676000,17750000
...,...,...,...,...,...,...,...,...,...,...,...
65,1201890,21137000.0,2718000.0,15149000.0,3270000,1814000,1348000,674000,853000,2083000,706000
66,1201920,595156000.0,69002000.0,255682000.0,270472000,203540000,37538000,4455000,36135000,50429000,16515000
67,1201950,44700000.0,4754000.0,28484000.0,11462000,9449000,1768000,1243000,2650000,5404000,2823000
68,1201980,101614000.0,8332000.0,16320000.0,76962000,70781000,5280000,1007000,5026000,9890000,4787000


# fl_saipe_poverity DataFrame

In [6]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/saipe/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_saipe_poverty_response = requests.get(query_url)
fl_saipe_poverty_data = fl_saipe_poverty_response.json()

# Establish the results from the request to loop through below
results_poverity = fl_saipe_poverty_data['results']

# Create Lists to store values for the columns
pov_nces_id = []
est_pop_total = []
est_pop_5to17 = []
est_pop_5to17_perc = []
est_pop_5to17_poverity = []
est_pop_5to17_poverity_perc = []


# Loop through results and store values in appropriate lists
for result in results_poverity:
    pov_nces_id.append(result['leaid'])
    est_pop_total.append(result['est_population_total'])
    est_pop_5to17.append(result['est_population_5_17'])
    est_pop_5to17_perc.append(result['est_population_5_17_pct'])
    est_pop_5to17_poverity.append(result['est_population_5_17_poverty'])
    est_pop_5to17_poverity_perc.append(result['est_population_5_17_poverty_pct'])

    
# Create dataframe from results
fl_saipe_poverity = pd.DataFrame({"NCES ID":pov_nces_id, "Estimated Total Population":est_pop_total, 
                                 "Estimated Total Population Ages 5–17":est_pop_5to17,                        
                                 "Share of Population that are school ages 5–17":est_pop_5to17_perc, 
                                 "Estimated Population Ages 5–17 in poverity":est_pop_5to17_poverity,
                                "Share of School-Age Population (5–17) in poverity":est_pop_5to17_poverity_perc})

# Format columns
# clean_fl_saipe_poverity = fl_saipe_poverity.style.format({"Estimated Total Population":"{:,.0f}", 
#                                                          "Estimated Total Population Ages 5–17":"{:,.0f}",                        
#                                                          "Share of Population that are school ages 5–17":"{:.2%}", 
#                                                          "Estimated Population Ages 5–17 in poverity":"{:,.0f}",
#                                                          "Share of School-Age Population (5–17) in poverity":"{:.2%}"
#                                                         })

fl_saipe_poverity

,NCES ID,Estimated Total Population,Estimated Total Population Ages 5–17,Share of Population that are school ages 5–17,Estimated Population Ages 5–17 in poverity,Share of School-Age Population (5–17) in poverity
0,1200030,256380.0,32012.0,0.124862,6438.0,0.201112
1,1200060,27093.0,5004.0,0.184697,1146.0,0.229017
2,1200090,178985.0,27626.0,0.154348,5939.0,0.214979
3,1200120,26702.0,3847.0,0.144072,1047.0,0.272160
4,1200150,556885.0,78183.0,0.140393,15431.0,0.197370
...,...,...,...,...,...,...
62,1201890,15190.0,2125.0,0.139895,510.0,0.240000
63,1201920,507531.0,67908.0,0.133801,16530.0,0.243418
64,1201950,31432.0,4887.0,0.155478,926.0,0.189482
65,1201980,61530.0,9060.0,0.147245,2256.0,0.249007


# fl_nces_combined DataFrame

In [7]:
# Combined all above API calls in to one dataframe to match with District Name

dem_pov_nces_combined = pd.merge(clean_fl_ccd_directory,fl_saipe_poverity, on="NCES ID")
fl_nces_combined = pd.merge(dem_pov_nces_combined,clean_fl_ccd_financial, on="NCES ID")
fl_nces_combined

,District Name,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,English Language Students,Total FTE Teachers,...,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
0,ALACHUA,1200030,01,12,1,73,28689,3788,584,1833,...,272451000.0,34881000.0,119810000.0,117760000,103083000,21763000,3041000,13128000,29535000,11174000
1,BAKER,1200060,02,32,1,9,4936,538,11,301,...,42674000.0,4704000.0,29197000.0,8773000,6266000,2595000,732000,2007000,4385000,2613000
2,BAY,1200090,03,22,1,51,27641,4575,567,1844,...,268131000.0,26772000.0,103527000.0,137832000,101316000,11183000,2045000,13550000,20567000,8132000
3,BRADFORD,1200120,04,32,1,12,3199,645,6,242,...,31053000.0,4941000.0,17021000.0,9091000,6476000,1490000,734000,1792000,3197000,1629000
4,BREVARD,1200150,05,21,1,129,72285,11931,2344,4776,...,673236000.0,76493000.0,320008000.0,276735000,221801000,49626000,5282000,36220000,55676000,17750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,UNION,1201890,63,32,1,9,2384,387,0,175,...,21137000.0,2718000.0,15149000.0,3270000,1814000,1348000,674000,853000,2083000,706000
63,VOLUSIA,1201920,64,21,1,92,61777,9879,3479,4153,...,595156000.0,69002000.0,255682000.0,270472000,203540000,37538000,4455000,36135000,50429000,16515000
64,WAKULLA,1201950,65,42,1,15,5141,976,8,327,...,44700000.0,4754000.0,28484000.0,11462000,9449000,1768000,1243000,2650000,5404000,2823000
65,WALTON,1201980,66,41,1,23,8435,799,300,569,...,101614000.0,8332000.0,16320000.0,76962000,70781000,5280000,1007000,5026000,9890000,4787000


# Dropout Data

In [8]:
# Read in Florida District Dropout Data
fl_2014_dropout_path = "fldoe_data/2014_FL_dropout_rates.xls"
fl_2014_dropout_data = pd.read_excel(os.path.join(fl_2014_dropout_path),header=4)

# Remove all district names that were dropped due to not local school district and state total
clean_fl_2014_dropout=fl_2014_dropout_data[~fl_2014_dropout_data["District Name"].isin(drop_ids)]
clean_fl_2014_dropout = clean_fl_2014_dropout.loc[clean_fl_2014_dropout['District Name'] != "FLORIDA"]

clean_fl_2014_dropout

,Dist #,District Name,White Dropouts (numerator),White Enrollment (denominator),White Dropout Rate,Black Dropouts (numerator),Black Enrollment (denominator),Black Dropout Rate,Hispanic Dropouts (numerator),Hispanic Enrollment (denominator),...,Pacific Islander Dropout Rate,American Indian Dropouts (numerator),American Indian Enrollment (denominator),American Indian Dropout Rate,Two or More Races Dropouts (numerator),Two or More Races Enrollment (denominator),Two or More Races Dropout Rate,Total Dropouts (numerator),Total Enrollment (denominator),Total Dropout Rate
1,1,ALACHUA,58,4418,0.0131281,174,3115,0.055859,17,803,...,*,1,20,0.05,14,419,0.0334129,265,9182,0.028861
2,2,BAKER,13,1131,0.0114943,0,151,0.000000,0,24,...,*,*,*,*,0,26,0,13,1344,0.009673
3,3,BAY,65,5713,0.0113776,16,1255,0.012749,8,461,...,*,0,34,0,4,359,0.0111421,95,8033,0.011826
4,4,BRADFORD,2,686,0.00291545,4,210,0.019048,0,24,...,*,*,*,*,0,24,0,6,955,0.006283
5,5,BREVARD,101,14766,0.00684004,48,3130,0.015335,26,3138,...,0,0,55,0,15,1441,0.0104094,191,23071,0.008279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,UNION,3,554,0.00541516,3,115,0.026087,0,36,...,*,*,*,*,0,28,0,6,741,0.008097
64,64,VOLUSIA,109,12604,0.00864805,58,3393,0.017094,32,3757,...,0,4,78,0.0512821,5,734,0.00681199,208,20976,0.009916
65,65,WAKULLA,16,1194,0.0134003,2,157,0.012739,0,37,...,*,*,*,*,0,61,0,18,1468,0.012262
66,66,WALTON,39,1870,0.0208556,3,208,0.014423,12,193,...,*,0,11,0,0,103,0,54,2408,0.022425


In [11]:
florida_dropout_rate = clean_fl_2014_dropout[["District Name", "Total Dropout Rate"]]
florida_dropout_rate.head()

,District Name,Total Dropout Rate
1,ALACHUA,0.028861
2,BAKER,0.009673
3,BAY,0.011826
4,BRADFORD,0.006283
5,BREVARD,0.008279


In [80]:
florida_dropout_rate.describe()

,Total Dropout Rate
count,67.000000
mean,0.016037
std,0.011796
min,0.000000
25%,0.007894
50%,0.011628
75%,0.022702
max,0.057082


# Dropout Data Mean

In [55]:
mean = florida_dropout_rate["Total Dropout Rate"].mean()
mean

0.016036537313432835

In [76]:
florida_dropout_rate_df = florida_dropout_rate.loc[(florida_dropout_rate["Total Dropout Rate"] > .011) & (florida_dropout_rate["Total Dropout Rate"] < .021)]

In [79]:
florida_dropout_rate_df = florida_dropout_rate.loc[(florida_dropout_rate["Total Dropout Rate"] > mean - .004) & (florida_dropout_rate["Total Dropout Rate"] < mean + .004)]
florida_dropout_rate_df

,District Name,Total Dropout Rate
6,BROWARD,0.014282
9,CITRUS,0.014999
19,FRANKLIN,0.014837
29,HILLSBOROUGH,0.013770
34,LAFAYETTE,0.014451
44,MONROE,0.013564
60,SUMTER,0.014013
65,WAKULLA,0.012262


# Dropout Data Median

In [60]:
median = florida_dropout_rate["Total Dropout Rate"].median()
median

0.011628

In [69]:
florida_dropout_rate_df = florida_dropout_rate.loc[(florida_dropout_rate["Total Dropout Rate"] > .011) & (florida_dropout_rate["Total Dropout Rate"] < .021)]
# high_low = ["above" if florida_dropout_rate.loc[florida_dropout_rate["Total Dropout Rate"] > median]]

In [90]:
florida_dropout_rate_df = florida_dropout_rate.loc[(florida_dropout_rate["Total Dropout Rate"] > median - .002) & (florida_dropout_rate["Total Dropout Rate"] < median + .003)]
florida_dropout_rate_df

,District Name,Total Dropout Rate
2,BAKER,0.009673
3,BAY,0.011826
6,BROWARD,0.014282
10,CLAY,0.010588
11,COLLIER,0.010949
17,ESCAMBIA,0.010126
22,GLADES,0.011628
23,GULF,0.009820
29,HILLSBOROUGH,0.013770
34,LAFAYETTE,0.014451


# Dropout Data Above Median Range

In [91]:
florida_dropout_rate_df_above = florida_dropout_rate_df.loc[florida_dropout_rate_df["Total Dropout Rate"] > median]
districts_above = florida_dropout_rate_df_above["District Name"]
districts_above

3              BAY
6          BROWARD
29    HILLSBOROUGH
34       LAFAYETTE
44          MONROE
60          SUMTER
65         WAKULLA
Name: District Name, dtype: object

# Dropout Data Below Median Range

In [94]:
florida_dropout_rate_df_below = florida_dropout_rate_df.loc[florida_dropout_rate_df["Total Dropout Rate"] < median]
districts_below = florida_dropout_rate_df_below["District Name"]
districts_below

2         BAKER
10         CLAY
11      COLLIER
17     ESCAMBIA
23         GULF
46     OKALOOSA
56    ST. LUCIE
58     SARASOTA
64      VOLUSIA
Name: District Name, dtype: object

# Demographics Analysis: